# JUICE RPWI HF SID7 (PSSR3 surv): L1a QL -- 2025/7/5

# Import lib

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# Library setting

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf

import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_1-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cdf_lib as juice_cdf
import juice_sid7_data as juice_data
import juice_sid7_lib as juice_sid7

# Mode seting

In [ ]:
# *** Plot dump ***
dump_mode = 0                           # 0: no-dump  1:plot dump
# *** CAL set ***
unit_mode = 0                           # 0: raw    1: dBm＠ADC  2: V@HF   3:V2@HF   4:V2@RWI
band_mode = 0                           # 0: sum    1: /Hz
cal_mode  = 2                           # [Power]     0: background   1: CAL     2: all
# *** Unit set ***
time_mode = 0                           # [Time]      0: Epoch   1: data number
gap_mode  = 0                           # [gap]       0: n/a     1: gap
# *** Parameter set ***
p_raw_max = 8.5                         # background: 7.5   CAL: 10
p_raw_min = 1.0                         # background: 2.5   CAL: 5
# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder

In [ ]:
f_min     = 0000                        # [Frequency sel]   max - min
f_max     = 45000                       # [Frequency sel]   max - min

# get CDF data

In [ ]:
# *** Flight data: Ver.2 ***
# date='20240126';  ver = 'V02'
# date='20240706';  ver = 'V02'
# date='20240819';  ver = 'V02'
# date='20240821';  ver = 'V02'

# *** Ground test data ***
date='0';  ver = 'V02'

data_dir, data_name_list = juice_data.datalist(date, ver)       # [date]   yyyymmdd: group read    others: file list

In [ ]:
class struct:
    pass

data = struct()
num_list = len(data_name_list)
for i in range(num_list):
    data_name = data_name_list[i];  cdf_file = data_dir + data_name;  print(i, cdf_file)
    # RPWI_FSW_version = juice_cdf._RPWI_FSW_version(cdf_file)
    cdf = pycdf.CDF(cdf_file);      data1 = juice_sid7.hf_sid7_read(cdf) #  , RPWI_FSW_version)
    if i==0:
        data = data1;                                  print(data.auto_corr)
    else:
        data = juice_sid23.hf_sid23_add(data, data1);  print(data.auto_corr)
data_name = os.path.split(data_name)[1];               print("data name:", data_name)

In [ ]:
data = juice_sid7.hf_sid7_shaping(data, f_max, f_min)
date1 = data.epoch[0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]; date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2;  print(str_date)
n_time0 = data.n_time - 1
freq_center = data.freq_center[0]

In [ ]:
if data.n_time>1:
    date3 = data.epoch[1];  date3 = date3.strftime('%Y/%m/%d %R:%S')
    print("       date and time:", str_date, "(interval:", data.epoch[1] - data.epoch[0], date3, ")")
else:
    print("       date and time:", str_date)
print("data size:", data.auto_corr.shape, " frequency_center:", freq_center, "[kHz]")

# Raw data

In [ ]:
fig = plt.figure(figsize=(16, 11))
ax1 = fig.add_subplot(5, 1, 1);  ax2 = fig.add_subplot(5, 1, 2);  ax3 = fig.add_subplot(5, 1, 3)
ax4 = fig.add_subplot(5, 1, 4);  ax5 = fig.add_subplot(5, 1, 5)

ax1.plot(np.ravel(data.E_i[:][:]),   '-r',    linewidth=.5, label='E_i')
ax1.plot(np.ravel(data.E_q[:][:]),   '-g',    linewidth=.5, label='E_q')
ax2.plot(np.ravel(data.auto_corr[:][:]),      linewidth=.5, label='auto_corr')
ax3.plot(np.ravel(data.time[:][:]),           linewidth=.5, label='time')
ax4.plot(np.ravel(data.time_block[:][:]), linewidth=.5, label='time_autocorr')
ax5.plot(np.ravel(data.epoch[:]), '.')
ax1.set_ylabel('E_i & E_q')
ax2.set_ylabel('AutoCorr')
ax3.set_ylabel('time [sec]')
ax4.set_ylabel('time_auto_corr [sec]')
ax5.set_ylabel('Date-Time')
ax5.set_xlabel(str_date)
#
title_label = '[JUICE/RPWI HF PSSR3-Surv (SID-7)]\n' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8); 
# xlim=[-.5, len(np.ravel(data.auto_corr)) -.5]; print(xlim); ax1.set_xlim(xlim)
# xlim=[-.5, len(np.ravel(data.epoch))     -.5]; print(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim)
# ylim=[10**p_raw_min, 10**p_raw_max];  ax1.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

# First data

In [ ]:
n_sweep = 0

fig = plt.figure(figsize=(16, 11))
ax1 = fig.add_subplot(4, 4, 1);   ax2 = fig.add_subplot(4, 4, 2);   ax3 = fig.add_subplot(4, 4, 3);   ax4 = fig.add_subplot(4, 4, 4)
ax5 = fig.add_subplot(4, 4, 5);   ax6 = fig.add_subplot(4, 4, 6);   ax7 = fig.add_subplot(4, 4, 7);   ax8 = fig.add_subplot(4, 4, 8)
ax9 = fig.add_subplot(4, 4, 9);   ax10 = fig.add_subplot(4, 4, 10); ax11 = fig.add_subplot(4, 4, 11); ax12 = fig.add_subplot(4, 4, 12)
ax13 = fig.add_subplot(4, 4, 13); ax14 = fig.add_subplot(4, 4, 14); ax15 = fig.add_subplot(4, 4, 15)

ax1.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][0][:], '-r', linewidth=.5, label='0')
ax1.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][1][:], '-g', linewidth=.5, label='1')
ax1.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][2][:], '-b', linewidth=.5, label='2')
ax2.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][3][:], '-r', linewidth=.5, label='3')
ax2.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][4][:], '-g', linewidth=.5, label='4')
ax2.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][5][:], '-b', linewidth=.5, label='5')
ax3.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][6][:], '-r', linewidth=.5, label='6')
ax3.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][7][:], '-g', linewidth=.5, label='7')
ax3.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][8][:], '-b', linewidth=.5, label='8')
ax4.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][9][:], '-r', linewidth=.5, label='9')
ax4.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][10][:], '-g', linewidth=.5, label='10')
ax4.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][11][:], '-b', linewidth=.5, label='11')
ax5.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][12][:], '-r', linewidth=.5, label='12')
ax5.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][13][:], '-g', linewidth=.5, label='13')
ax5.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][14][:], '-b', linewidth=.5, label='14')
ax6.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][15][:], '-r', linewidth=.5, label='15')
ax6.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][16][:], '-g', linewidth=.5, label='16')
ax6.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][17][:], '-b', linewidth=.5, label='17')
ax7.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][18][:], '-r', linewidth=.5, label='18')
ax7.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][19][:], '-g', linewidth=.5, label='19')
ax7.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][20][:], '-b', linewidth=.5, label='20')
ax8.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][21][:], '-r', linewidth=.5, label='21')
ax8.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][22][:], '-g', linewidth=.5, label='22')
ax8.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][23][:], '-b', linewidth=.5, label='23')
ax9.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][24][:], '-r', linewidth=.5, label='24')
ax9.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][25][:], '-g', linewidth=.5, label='25')
ax9.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][26][:], '-b', linewidth=.5, label='26')
ax10.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][27][:], '-r', linewidth=.5, label='27')
ax10.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][28][:], '-g', linewidth=.5, label='28')
ax10.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][29][:], '-b', linewidth=.5, label='29')
ax11.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][30][:], '-r', linewidth=.5, label='30')
ax11.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][31][:], '-g', linewidth=.5, label='31')
ax11.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][32][:], '-b', linewidth=.5, label='32')
ax12.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][33][:], '-r', linewidth=.5, label='33')
ax12.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][34][:], '-g', linewidth=.5, label='34')
ax12.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][35][:], '-b', linewidth=.5, label='35')
ax13.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][36][:], '-r', linewidth=.5, label='36')
ax13.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][37][:], '-g', linewidth=.5, label='37')
ax13.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][38][:], '-b', linewidth=.5, label='38')
ax14.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][39][:], '-r', linewidth=.5, label='39')
ax14.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][40][:], '-g', linewidth=.5, label='40')
ax14.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][41][:], '-b', linewidth=.5, label='41')
ax15.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][42][:], '-r', linewidth=.5, label='42')
ax15.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][43][:], '-g', linewidth=.5, label='43')
ax15.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][44][:], '-b', linewidth=.5, label='44')

date1 = data.epoch[n_sweep];  date1 = date1.strftime('First: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]\n" + date1;  ax2.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);   ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);   ax6.legend(loc='upper right', fontsize=8)
ax7.legend(loc='upper right', fontsize=8);  ax8.legend(loc='upper right', fontsize=8);   ax9.legend(loc='upper right', fontsize=8)
ax10.legend(loc='upper right', fontsize=8); ax11.legend(loc='upper right', fontsize=8);  ax12.legend(loc='upper right', fontsize=8)
ax10.legend(loc='upper right', fontsize=8); ax11.legend(loc='upper right', fontsize=8);  ax12.legend(loc='upper right', fontsize=8)
ax13.legend(loc='upper right', fontsize=8); ax14.legend(loc='upper right', fontsize=8);  ax15.legend(loc='upper right', fontsize=8)
ax12.set_xlabel("Time [sec]"); ax13.set_xlabel("Time [sec]"); ax14.set_xlabel("Time [sec]"); ax15.set_xlabel("Time [sec]")

fig.subplots_adjust(hspace=0)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_1st.png'
    fig.savefig(png_fname)

## Mid data

In [ ]:
#n_sweep = np.int16(n_time0/2)
n_sweep = n_time0-1
print("[mid]", n_sweep, "in total:", n_time0)

fig = plt.figure(figsize=(16, 11))
ax1 = fig.add_subplot(4, 4, 1);   ax2 = fig.add_subplot(4, 4, 2);   ax3 = fig.add_subplot(4, 4, 3);   ax4 = fig.add_subplot(4, 4, 4)
ax5 = fig.add_subplot(4, 4, 5);   ax6 = fig.add_subplot(4, 4, 6);   ax7 = fig.add_subplot(4, 4, 7);   ax8 = fig.add_subplot(4, 4, 8)
ax9 = fig.add_subplot(4, 4, 9);   ax10 = fig.add_subplot(4, 4, 10); ax11 = fig.add_subplot(4, 4, 11); ax12 = fig.add_subplot(4, 4, 12)
ax13 = fig.add_subplot(4, 4, 13); ax14 = fig.add_subplot(4, 4, 14); ax15 = fig.add_subplot(4, 4, 15)

ax1.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][0][:], '-r', linewidth=.5, label='0')
ax1.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][1][:], '-g', linewidth=.5, label='1')
ax1.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][2][:], '-b', linewidth=.5, label='2')
ax2.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][3][:], '-r', linewidth=.5, label='3')
ax2.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][4][:], '-g', linewidth=.5, label='4')
ax2.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][5][:], '-b', linewidth=.5, label='5')
ax3.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][6][:], '-r', linewidth=.5, label='6')
ax3.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][7][:], '-g', linewidth=.5, label='7')
ax3.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][8][:], '-b', linewidth=.5, label='8')
ax4.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][9][:], '-r', linewidth=.5, label='9')
ax4.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][10][:], '-g', linewidth=.5, label='10')
ax4.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][11][:], '-b', linewidth=.5, label='11')
ax5.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][12][:], '-r', linewidth=.5, label='12')
ax5.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][13][:], '-g', linewidth=.5, label='13')
ax5.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][14][:], '-b', linewidth=.5, label='14')
ax6.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][15][:], '-r', linewidth=.5, label='15')
ax6.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][16][:], '-g', linewidth=.5, label='16')
ax6.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][17][:], '-b', linewidth=.5, label='17')
ax7.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][18][:], '-r', linewidth=.5, label='18')
ax7.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][19][:], '-g', linewidth=.5, label='19')
ax7.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][20][:], '-b', linewidth=.5, label='20')
ax8.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][21][:], '-r', linewidth=.5, label='21')
ax8.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][22][:], '-g', linewidth=.5, label='22')
ax8.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][23][:], '-b', linewidth=.5, label='23')
ax9.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][24][:], '-r', linewidth=.5, label='24')
ax9.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][25][:], '-g', linewidth=.5, label='25')
ax9.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][26][:], '-b', linewidth=.5, label='26')
ax10.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][27][:], '-r', linewidth=.5, label='27')
ax10.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][28][:], '-g', linewidth=.5, label='28')
ax10.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][29][:], '-b', linewidth=.5, label='29')
ax11.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][30][:], '-r', linewidth=.5, label='30')
ax11.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][31][:], '-g', linewidth=.5, label='31')
ax11.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][32][:], '-b', linewidth=.5, label='32')
ax12.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][33][:], '-r', linewidth=.5, label='33')
ax12.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][34][:], '-g', linewidth=.5, label='34')
ax12.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][35][:], '-b', linewidth=.5, label='35')
ax13.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][36][:], '-r', linewidth=.5, label='36')
ax13.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][37][:], '-g', linewidth=.5, label='37')
ax13.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][38][:], '-b', linewidth=.5, label='38')
ax14.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][39][:], '-r', linewidth=.5, label='39')
ax14.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][40][:], '-g', linewidth=.5, label='40')
ax14.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][41][:], '-b', linewidth=.5, label='41')
ax15.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][42][:], '-r', linewidth=.5, label='42')
ax15.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][43][:], '-g', linewidth=.5, label='43')
ax15.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][44][:], '-b', linewidth=.5, label='44')

date1 = data.epoch[n_sweep];  date1 = date1.strftime('First: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]\n" + date1;  ax2.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);   ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);   ax6.legend(loc='upper right', fontsize=8)
ax7.legend(loc='upper right', fontsize=8);  ax8.legend(loc='upper right', fontsize=8);   ax9.legend(loc='upper right', fontsize=8)
ax10.legend(loc='upper right', fontsize=8); ax11.legend(loc='upper right', fontsize=8);  ax12.legend(loc='upper right', fontsize=8)
ax10.legend(loc='upper right', fontsize=8); ax11.legend(loc='upper right', fontsize=8);  ax12.legend(loc='upper right', fontsize=8)
ax13.legend(loc='upper right', fontsize=8); ax14.legend(loc='upper right', fontsize=8);  ax15.legend(loc='upper right', fontsize=8)
ax12.set_xlabel("Time [sec]"); ax13.set_xlabel("Time [sec]"); ax14.set_xlabel("Time [sec]"); ax15.set_xlabel("Time [sec]")

fig.subplots_adjust(hspace=0)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_mid.png'
    fig.savefig(png_fname)

# Last data

In [ ]:
n_sweep = n_time0 - 1

fig = plt.figure(figsize=(16, 11))
ax1 = fig.add_subplot(4, 4, 1);   ax2 = fig.add_subplot(4, 4, 2);   ax3 = fig.add_subplot(4, 4, 3);   ax4 = fig.add_subplot(4, 4, 4)
ax5 = fig.add_subplot(4, 4, 5);   ax6 = fig.add_subplot(4, 4, 6);   ax7 = fig.add_subplot(4, 4, 7);   ax8 = fig.add_subplot(4, 4, 8)
ax9 = fig.add_subplot(4, 4, 9);   ax10 = fig.add_subplot(4, 4, 10); ax11 = fig.add_subplot(4, 4, 11); ax12 = fig.add_subplot(4, 4, 12)
ax13 = fig.add_subplot(4, 4, 13); ax14 = fig.add_subplot(4, 4, 14); ax15 = fig.add_subplot(4, 4, 15)

ax1.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][0][:], '-r', linewidth=.5, label='0')
ax1.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][1][:], '-g', linewidth=.5, label='1')
ax1.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][2][:], '-b', linewidth=.5, label='2')
ax2.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][3][:], '-r', linewidth=.5, label='3')
ax2.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][4][:], '-g', linewidth=.5, label='4')
ax2.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][5][:], '-b', linewidth=.5, label='5')
ax3.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][6][:], '-r', linewidth=.5, label='6')
ax3.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][7][:], '-g', linewidth=.5, label='7')
ax3.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][8][:], '-b', linewidth=.5, label='8')
ax4.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][9][:], '-r', linewidth=.5, label='9')
ax4.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][10][:], '-g', linewidth=.5, label='10')
ax4.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][11][:], '-b', linewidth=.5, label='11')
ax5.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][12][:], '-r', linewidth=.5, label='12')
ax5.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][13][:], '-g', linewidth=.5, label='13')
ax5.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][14][:], '-b', linewidth=.5, label='14')
ax6.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][15][:], '-r', linewidth=.5, label='15')
ax6.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][16][:], '-g', linewidth=.5, label='16')
ax6.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][17][:], '-b', linewidth=.5, label='17')
ax7.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][18][:], '-r', linewidth=.5, label='18')
ax7.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][19][:], '-g', linewidth=.5, label='19')
ax7.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][20][:], '-b', linewidth=.5, label='20')
ax8.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][21][:], '-r', linewidth=.5, label='21')
ax8.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][22][:], '-g', linewidth=.5, label='22')
ax8.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][23][:], '-b', linewidth=.5, label='23')
ax9.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][24][:], '-r', linewidth=.5, label='24')
ax9.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][25][:], '-g', linewidth=.5, label='25')
ax9.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][26][:], '-b', linewidth=.5, label='26')
ax10.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][27][:], '-r', linewidth=.5, label='27')
ax10.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][28][:], '-g', linewidth=.5, label='28')
ax10.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][29][:], '-b', linewidth=.5, label='29')
ax11.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][30][:], '-r', linewidth=.5, label='30')
ax11.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][31][:], '-g', linewidth=.5, label='31')
ax11.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][32][:], '-b', linewidth=.5, label='32')
ax12.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][33][:], '-r', linewidth=.5, label='33')
ax12.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][34][:], '-g', linewidth=.5, label='34')
ax12.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][35][:], '-b', linewidth=.5, label='35')
ax13.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][36][:], '-r', linewidth=.5, label='36')
ax13.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][37][:], '-g', linewidth=.5, label='37')
ax13.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][38][:], '-b', linewidth=.5, label='38')
ax14.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][39][:], '-r', linewidth=.5, label='39')
ax14.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][40][:], '-g', linewidth=.5, label='40')
ax14.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][41][:], '-b', linewidth=.5, label='41')
ax15.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][42][:], '-r', linewidth=.5, label='42')
ax15.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][43][:], '-g', linewidth=.5, label='43')
ax15.plot(data.time[n_sweep][0][:], data.auto_corr[n_sweep][44][:], '-b', linewidth=.5, label='44')

date1 = data.epoch[n_sweep];  date1 = date1.strftime('First: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]\n" + date1;  ax2.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);   ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);   ax6.legend(loc='upper right', fontsize=8)
ax7.legend(loc='upper right', fontsize=8);  ax8.legend(loc='upper right', fontsize=8);   ax9.legend(loc='upper right', fontsize=8)
ax10.legend(loc='upper right', fontsize=8); ax11.legend(loc='upper right', fontsize=8);  ax12.legend(loc='upper right', fontsize=8)
ax10.legend(loc='upper right', fontsize=8); ax11.legend(loc='upper right', fontsize=8);  ax12.legend(loc='upper right', fontsize=8)
ax13.legend(loc='upper right', fontsize=8); ax14.legend(loc='upper right', fontsize=8);  ax15.legend(loc='upper right', fontsize=8)
ax12.set_xlabel("Time [sec]"); ax13.set_xlabel("Time [sec]"); ax14.set_xlabel("Time [sec]"); ax15.set_xlabel("Time [sec]")


fig.subplots_adjust(hspace=0)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_last.png'
    fig.savefig(png_fname)

## Overall

In [ ]:
n_num = data.n_time * data.n_block
num_1d = np.arange(n_num)
auto_corr_2d = np.ravel(data.auto_corr)
auto_corr_2d = auto_corr_2d.reshape(n_num, data.n_lag).transpose()

fig2d = plt.figure(figsize=[16,11])
ax1 = fig2d.add_subplot(1, 1, 1); 

p1 = ax1.pcolormesh(num_1d, data.time[0][0], auto_corr_2d, cmap='bwr')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical")
ax1.set_title("[" + data_name + "]")

# Plot
fig2d.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_overall.png'
    fig2d.savefig(png_fname)